In [1]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:

import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import os




In [3]:
class TFRecordVectorDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.paths = self.data['path'].tolist()
        self.labels = self.data.iloc[:, -14:].values.astype(np.float32)  # labels are in the last 14 columns

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        try:
            # readin .tfrecord
            npy_path = path.replace('.tfrecord', '.npy')
            embedding = np.load(npy_path)

            
            # Convert to Tensor
            embedding_tensor = torch.tensor(embedding, dtype=torch.float32)  
            label_tensor = torch.tensor(self.labels[idx], dtype=torch.float32)  
            return embedding_tensor, label_tensor
        except Exception as e:
            print(f"Error loading {path}: {str(e)}")
            return torch.zeros(1376), torch.zeros(14)  

In [4]:
# Load csv file

# Use the L40S GPU
os.environ['CUDA_VISIBLE_DEVICES'] = "5"
print(os.getenv('CUDA_VISIBLE_DEVICES'))

# load  training set and test set
#train_df, test_df = train_test_split(data, test_size=0.2, random_state=39)
train_df = pd.read_csv("preprocessed_training.csv")
test_df = pd.read_csv("preprocessed_test.csv")



train_dataset = TFRecordVectorDataset(train_df)
test_dataset = TFRecordVectorDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)



5


In [5]:
print(train_df.shape)
print("\nlabel distribution in training set:")
print(train_df.select_dtypes(include=[np.number]).sum())

(207314, 33)

label distribution in training set:
gender                                112018.0
insurance_Medicaid                     17152.0
insurance_Medicare                     92265.0
insurance_Other                        97897.0
20-30                                   9882.0
30-40                                  12025.0
40-50                                  22666.0
50-60                                  40674.0
60-80                                  87345.0
80+                                    33812.0
race_AMERICAN INDIAN/ALASKA NATIVE       480.0
race_ASIAN                              6630.0
race_BLACK/AFRICAN AMERICAN            32887.0
race_HISPANIC/LATINO                   11035.0
race_OTHER                              9664.0
race_UNABLE TO OBTAIN                    918.0
race_UNKNOWN                            9844.0
race_WHITE                            135856.0
Enlarged Cardiomediastinum              6770.0
Cardiomegaly                           42720.0
Lung Opaci

In [6]:
print(test_df.shape)
print("\nlabel distribution in test set:")
print(test_df.select_dtypes(include=[np.number]).sum())

(21591, 31)

label distribution in test set:
gender                                11913.0
insurance_Medicaid                     1812.0
insurance_Medicare                     9862.0
insurance_Other                        9917.0
20-30                                  1067.0
30-40                                  1238.0
40-50                                  2425.0
50-60                                  4103.0
60-80                                  9189.0
80+                                    3487.0
race_AMERICAN INDIAN/ALASKA NATIVE      184.0
race_ASIAN                              760.0
race_BLACK/AFRICAN AMERICAN            3685.0
race_HISPANIC/LATINO                   1392.0
race_OTHER                             1062.0
race_WHITE                            14508.0
Enlarged Cardiomediastinum              761.0
Cardiomegaly                           4441.0
Lung Opacity                           4904.0
Lung Lesion                             634.0
Edema                              

In [7]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_size, hidden_size1)
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        # Output
        self.fc3 = nn.Linear(hidden_size2, output_size)
        # Activation Func
        self.relu = nn.ReLU()

    def forward(self, x):
        # First
        out = self.fc1(x)
        out = self.relu(out)
        # Second
        out = self.fc2(out)
        out = self.relu(out)
        # Output
        out = self.fc3(out)
        return out

In [8]:

# Initialization
input_size = 1376  # dimension of input vector
hidden_size1 = 512  # dimension of first layer 
hidden_size2 = 256  # dimension of second layer
output_size = 14   # number of labels


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = MLP(input_size, hidden_size1, hidden_size2, output_size)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)
# optimizer and loss function
criterion = nn.BCEWithLogitsLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

cuda


In [9]:
from sklearn.metrics import roc_auc_score
import numpy as np
from tqdm import tqdm

# Define number epochs
num_epochs = 20
best_auroc = 0.0

In [10]:
for epoch in range(num_epochs):
    # training
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Training'):
        inputs, labels = inputs.to(device), labels.to(device)
        #inputs, labels = inputs.cuda(), labels.cuda()
        

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    

    train_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}')

    # evaluation
    model.eval()
    all_probs = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Validation'):
            inputs, labels = inputs.to(device), labels.to(device)

            

            outputs = model(inputs)
            probs = torch.sigmoid(outputs) 
            
            # save results
            all_probs.append(probs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    # calculate AUROC
    all_probs = np.concatenate(all_probs, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    auroc_scores = []
    for i in range(all_labels.shape[1]):  # triverse all labels
        try:
            auroc = roc_auc_score(all_labels[:, i], all_probs[:, i])
            auroc_scores.append(auroc)
        except ValueError:
            print(f"Label {i} has no positive or negative samples in the test set.")
            auroc_scores.append(np.nan)
    
    # calculate macro AUROC
    macro_auroc = np.nanmean(auroc_scores)
    print(f'Epoch {epoch+1}, Validation Macro AUROC: {macro_auroc:.4f}')

    # save the best model
    if macro_auroc > best_auroc:
        best_auroc = macro_auroc
        torch.save(model.state_dict(), 'best_mlp_model.pth')
        print(f'Best model saved with AUROC: {best_auroc:.4f}')

print('Training complete.')

Epoch 1/20 - Training: 100%|███████████████████████████████████████████████████████| 6479/6479 [00:55<00:00, 116.29it/s]


Epoch 1, Train Loss: 0.2593


Epoch 1/20 - Validation: 100%|███████████████████████████████████████████████████████| 675/675 [00:03<00:00, 176.45it/s]


Epoch 1, Validation Macro AUROC: 0.8153
Best model saved with AUROC: 0.8153


Epoch 2/20 - Training: 100%|███████████████████████████████████████████████████████| 6479/6479 [00:51<00:00, 125.38it/s]


...

Epoch 19, Validation Macro AUROC: 0.8245


Epoch 20/20 - Training: 100%|██████████████████████████████████████████████████████| 6479/6479 [00:51<00:00, 126.63it/s]


Epoch 20, Train Loss: 0.2459


Epoch 20/20 - Validation: 100%|██████████████████████████████████████████████████████| 675/675 [00:03<00:00, 194.91it/s]


Epoch 20, Validation Macro AUROC: 0.8235
Training complete.


In [11]:
# Save results using the model trained after last epoch

column_name = [
    "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", "Lung Lesion",
    "Edema", "Consolidation", "Pneumonia", "Atelectasis", "Pneumothorax",
    "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices", "No Finding"
]
df_labels = pd.DataFrame(all_labels, columns=column_name)

df_labels.to_csv("true_labels_sub.csv", index=False)


In [12]:
df_predictions = pd.DataFrame(all_probs, columns=column_name)
df_predictions.to_csv("predicted_labels_sub.csv", index=False)